In [ ]:
%matplotlib widget

In [ ]:
from glob import glob
import numpy as np
import flammkuchen as fl
from split_dataset import SplitDataset
from bouter import Experiment
from scipy import stats
from lavian_et_al_2025.visual_motion.stimulus_functions import quantize_directions, make_sensory_regressors

import matplotlib.pyplot as plt
import colorspacious

import json
from pathlib import Path

from lavian_et_al_2025.imaging.imaging_functions import corr2_coeff
#from vision_and_navigation.imaging.visual_motion import *

In [ ]:
def JCh_to_RGB255(x):
    output = np.clip(colorspacious.cspace_convert(x, "JCh", "sRGB1"), 0, 1)
    return (output * 255).astype(np.uint8)


def color_stack(
        amp,
        angle,
        hueshift=2.5,
        amp_percentile=80,
        maxsat=50,
        lightness_min=100,
        lightness_delta=-40,
    ):
    output_lch = np.zeros((amp.shape[0], 3))
    output_lch[:, 0]
    maxamp = np.percentile(amp, amp_percentile)

    output_lch[:, 0] = (
            lightness_min + (np.clip(amp / maxamp, 0, 1)) * lightness_delta
    )
    output_lch[:, 1] = (np.clip(amp / maxamp, 0, 1)) * maxsat
    output_lch[:, 2] = (-angle + hueshift) * 180 / np.pi

    return JCh_to_RGB255(output_lch)

In [ ]:
# calculate directional tuning from zscored traces for each roi
def get_tuning_map(traces, sens_regs, n_dirs=8):

    n_t = sens_regs.shape[0]
    reg = sens_regs.values.T @ traces[:n_t, :]
    
    # tuning vector
    bin_centers, bins = quantize_directions([0], n_dirs)
    vectors = np.stack([np.cos(bin_centers), np.sin(bin_centers)], 0)
    reg_vectors = vectors @ reg
    
    angle = np.arctan2(reg_vectors[1], reg_vectors[0])
    amp = np.sqrt(np.sum(reg_vectors ** 2, 0))

    return amp, angle

# Set fish path

In [ ]:
master = Path(r'\\portulab.synology.me\data\Hagar and Ot\E0040\v10\LS ablation\{}'.format(treatment))
fish_list = list(master.glob("*_f*_{}_{}".format(treatment, session)))

treatment = 'control' #'control' #'ntr'
session = 'pre' #'post' #'pre'

# Load morphed coordinates

In [ ]:
morphed_coords = {}
in_brain_arr = {}

for fish in fish_list:
    print(fish)
    
    #Load morphed coords
    morphed_coords[fish.name] = fl.load(fish / 'registration' / 'to_h2b_baier_ref' / 'antspy' / 'mov_coords_transformed.h5')
    
    suite2p_brain = fl.load(fish / "data_from_suite2p_cells_brain.h5")
    
    in_brain_arr[fish.name] = np.full(morphed_coords[fish.name].shape[0], False)
    in_brain_arr[fish.name][suite2p_brain['coords_idx']] = True
    
print('Done')

In [ ]:
#And pool in a single array
coords_pooled = np.concatenate([morphed_coords[fish.name] for fish in fish_list], 0)

# Sensory regressors

In [ ]:
titles = ['right', 'backward right', 'backward', 'backward left', 'left', 'forward left', 'forward', 'forward right', ]
plot_dir = 0

In [ ]:
try:
    reg_corrcoefs_pooled = fl.load(master / 'reg_corrcoefs_pooled_{}_{}_dir{}.h5'.format(treatment, session, plot_dir))
    
except OSError:
    reg_corrcoefs = []

    for path in fish_list:
        traces = fl.load(path / "filtered_traces.h5", "/detr")
        sensory_regressors = fl.load(path / "sensory_regressors.h5", "/regressors")

        current_dir = np.asarray(sensory_regressors.iloc[:, plot_dir])        
        reg_corrcoefs.append(corr2_coeff(traces.T, current_dir.reshape(1, -1)).ravel())

    reg_corrcoefs_pooled = np.concatenate(reg_corrcoefs)
    fl.save(master / 'reg_corrcoefs_pooled_{}_{}_dir{}.h5'.format(treatment, session, plot_dir), reg_corrcoefs_pooled)

In [ ]:
in_brain_arr_pooled = np.concatenate([in_brain_arr[fish.name] for fish in fish_list])

In [ ]:
#Filter ROIs and normalize vector amplitude
coords_ib_pooled = coords_pooled[in_brain_arr_pooled]
reg_corrcoefs_ib_pooled = reg_corrcoefs_pooled[in_brain_arr_pooled]
mp_ind_regressor = np.argsort(np.abs(reg_corrcoefs_ib_pooled))

In [ ]:
dot_s = 0.1

scale_bar_len = 100
scale_bar_xpos = 10
scale_bar_ypos1 = 650
fs = 8

In [ ]:
#Import reference anatomy [from first fish]
ref_anatomy = fl.load(fish_list[0] / 'registration' / 'to_h2b_baier_ref' / 'antspy' / 'ref_mapped.h5')

In [ ]:
x_crop = [30, 580]
y_crop = [40, 740]
z_crop = [0, ref_anatomy.shape[2]]

ref_anatomy_cropped = ref_anatomy[x_crop[0]:x_crop[1], y_crop[0]:y_crop[1], z_crop[0]:z_crop[1]]

## Generate correlation map plot

In [ ]:
y_main_rat = ref_anatomy_cropped.shape[1]/ref_anatomy_cropped.shape[0]
x_rat = ref_anatomy_cropped.shape[2]/ref_anatomy_cropped.shape[0]
y_rat = ref_anatomy_cropped.shape[2]/ref_anatomy_cropped.shape[1]*y_main_rat

fig = plt.figure(figsize=(4,4))
gs = fig.add_gridspec(2, 2, width_ratios=[1, x_rat], height_ratios=[y_rat, y_main_rat])

ax1 = fig.add_subplot(gs[1, 0])
ax2 = fig.add_subplot(gs[0, 0], sharex=ax1)
ax3 = fig.add_subplot(gs[1, 1], sharey=ax1)

mock_anatomy = np.zeros_like(ref_anatomy_cropped)
ax1.imshow(mock_anatomy.mean(2).T, cmap='gray_r')
ax2.imshow(mock_anatomy.mean(1).T, cmap='gray_r')
ax3.imshow(mock_anatomy.mean(0), cmap='gray_r')

ax1.scatter(coords_ib_pooled[mp_ind_regressor,0]-x_crop[0], coords_ib_pooled[mp_ind_regressor,1]-y_crop[0], c=reg_corrcoefs_ib_pooled[mp_ind_regressor], s=dot_s, alpha=0.8, cmap='coolwarm', vmin=-1, vmax=1)
ax2.scatter(coords_ib_pooled[mp_ind_regressor,0]-x_crop[0], coords_ib_pooled[mp_ind_regressor,2]-z_crop[0], c=reg_corrcoefs_ib_pooled[mp_ind_regressor], s=dot_s, alpha=0.8, cmap='coolwarm', vmin=-1, vmax=1)
ax3.scatter(coords_ib_pooled[mp_ind_regressor,2]-z_crop[0], coords_ib_pooled[mp_ind_regressor,1]-y_crop[0], c=reg_corrcoefs_ib_pooled[mp_ind_regressor], s=dot_s, alpha=0.8, cmap='coolwarm', vmin=-1, vmax=1)

#Scale bars
ax1.plot((scale_bar_xpos, scale_bar_xpos+scale_bar_len), (scale_bar_ypos1, scale_bar_ypos1), c='black')
ax1.text(scale_bar_xpos+(scale_bar_len/2), scale_bar_ypos1+10, r'{}$\mu$m'.format(scale_bar_len), va='top', ha='center', fontsize=fs)

for ax in [ax1, ax2, ax3]:
    ax.set_aspect('equal')
    ax.axis('off')
    
ax1.set_xlim([0, x_crop[1]-x_crop[0]])
ax1.set_ylim([0, y_crop[1]-y_crop[0]])

ax2.set_xlim([0, x_crop[1]-x_crop[0]])
ax2.set_ylim([0, z_crop[1]-z_crop[0]])

ax3.set_xlim([0, z_crop[1]-z_crop[0]])
ax3.set_ylim([0, y_crop[1]-y_crop[0]])

ax1.invert_yaxis()

plt.subplots_adjust(hspace=0.2, wspace=0.01, left=0.05, right=0.95, top=0.95, bottom=0.05)


## Generate tuning map plot

In [ ]:
try:
    tuning_arrs = fl.load(master / 'tuning_arrs_{}_{}.h5'.format(treatment, session))
    amp_pooled, angle_pooled = tuning_arrs['amp_pooled'], tuning_arrs['angle_pooled']

except OSError:
    amp_pooled = {}
    angle_pooled = {}

    for fish in fish_list:
        print(fish)

        #Load traces and stimulus metadata
        exp = glob(str(fish / "*behavior*"))[0]
        traces = fl.load(fish / "filtered_traces.h5", "/detr")

        fs = 2
        sampling = 1/fs
        time = np.linspace(0, traces.shape[0]*sampling, traces.shape[0])

        #Make list of sensory regressors 
        reg = make_sensory_regressors(Experiment(fish), sampling=sampling)
        reg_list = [reg]

        #Compute tuning
        amp_pooled[fish.name], angle_pooled[fish.name] = get_tuning_map(traces, reg)
        
    tuning_arrs = {'angle_pooled':angle_pooled, 'amp_pooled':amp_pooled}
    fl.save(master / 'tuning_arrs_{}_{}.h5'.format(treatment, session), tuning_arrs)

    print('Done')

In [ ]:
try:
    fish_source = fl.load(master / 'fish_source_{}_{}.h5'.format(treatment, session))

except OSError:
    
    fish_source_list = []
    
    for i, fish in enumerate(fish_list):
        fish_source_list.append(np.full((fl.load(fish / "filtered_traces.h5", "/detr").shape[1]), i))
        
    fish_source = np.concatenate(fish_source_list)
    fl.save(master / 'fish_source_{}_{}.h5'.format(treatment, session), fish_source)


In [ ]:
#Pool amp and angle arrays
amp_pooled_arr = np.concatenate([amp_pooled[fish.name] for fish in fish_list])
angle_pooled_arr = np.concatenate([angle_pooled[fish.name] for fish in fish_list])

#also boolean array to keep track of ROIs in the brain
in_brain_arr_pooled = np.concatenate([in_brain_arr[fish.name] for fish in fish_list])

In [ ]:
#Make filter to exclude NaNs from amp array (NaNs co-occur in amp and angle array)
nan_filt = np.isnan(amp_pooled_arr)
print(nan_filt.sum(), ' ROIs excluded')

#Combine into a filtering array
valid_rois = np.logical_and(in_brain_arr_pooled, ~nan_filt)

In [ ]:
#Color stack
colors_ib = color_stack(amp_pooled_arr[valid_rois], angle_pooled_arr[valid_rois])
# colors = color_stack(amp_pooled_arr, angle_pooled_arr)

In [ ]:
#Filter ROIs and normalize vector amplitude
coords_ib_pooled = coords_pooled[valid_rois]
amp_pooled_ib = amp_pooled_arr[valid_rois]
amp_norm = amp_pooled_ib / np.nanmax(amp_pooled_ib)

In [ ]:
#Load reliability arrays and filter ROIs
perct = 95

rel_arr_pooled = np.concatenate([fl.load(fish / "reliability_index_arr.h5", "/reliability_arr_combined") for fish in fish_list])
rel_thresh = np.percentile(rel_arr_pooled, perct) # 0.5
selected_vis = np.where(rel_arr_pooled[~nan_filt[in_brain_arr_pooled]] > rel_thresh)[0]

In [ ]:
## Generate a plot shoowing the contribution of each fish to the dataset

sel_roi_count = fish_source[valid_rois][selected_vis]
fish_i, roi_count = np.unique(sel_roi_count, return_counts=True)
fish_i, total_count = np.unique(fish_source, return_counts=True)

fig, ax = plt.subplots()
ax.bar(fish_i, total_count, color='white', edgecolor='black')
ax.bar(fish_i, roi_count, color='black')

ax.set_xticks(np.arange(len(fish_list)))
ax.set_xlabel('Fish')
ax.set_ylabel('# of ROIs')

plt.tight_layout()

In [ ]:
coords_vis_pooled = coords_ib_pooled[selected_vis]
colors_vis_pooled = colors_ib[selected_vis]
amp_vis = amp_pooled_ib[selected_vis]

mp_ind_pooled = np.argsort(amp_vis)

In [ ]:
dot_s = 1

scale_bar_len = 100
scale_bar_xpos = 100
scale_bar_ypos1 = 730
fs = 8

In [ ]:
#Plot
y_main_rat = ref_anatomy_cropped.shape[1]/ref_anatomy_cropped.shape[0]
x_rat = ref_anatomy_cropped.shape[2]/ref_anatomy_cropped.shape[0]
y_rat = ref_anatomy_cropped.shape[2]/ref_anatomy_cropped.shape[1]*y_main_rat

fig = plt.figure(figsize=(4,4))
gs = fig.add_gridspec(2, 2, width_ratios=[1, x_rat], height_ratios=[y_rat, y_main_rat])

ax1 = fig.add_subplot(gs[1, 0])
ax2 = fig.add_subplot(gs[0, 0], sharex=ax1)
ax3 = fig.add_subplot(gs[1, 1], sharey=ax1)

mock_anatomy = np.zeros_like(ref_anatomy_cropped)
ax1.imshow(mock_anatomy.mean(2).T, cmap='gray_r')
ax2.imshow(mock_anatomy.mean(1).T, cmap='gray_r')
ax3.imshow(mock_anatomy.mean(0), cmap='gray_r')

ax1.scatter(coords_ib_pooled[:,0]-x_crop[0], coords_ib_pooled[:,1]-y_crop[0], c='linen', s=dot_s/10, alpha=0.8)
ax1.scatter(coords_vis_pooled[mp_ind_pooled,0]-x_crop[0], coords_vis_pooled[mp_ind_pooled,1]-y_crop[0], c=colors_vis_pooled[mp_ind_pooled]/255, s=dot_s/10, alpha=0.8)

ax3.scatter(coords_ib_pooled[:,2]-z_crop[0], coords_ib_pooled[:,1]-y_crop[0], c='linen', s=dot_s/10, alpha=0.8)
ax3.scatter(coords_vis_pooled[mp_ind_pooled,2]-z_crop[0], coords_vis_pooled[mp_ind_pooled,1]-y_crop[0], c=colors_vis_pooled[mp_ind_pooled]/255, s=dot_s/10, alpha=0.8)

ax2.scatter(coords_ib_pooled[:,0]-x_crop[0], coords_ib_pooled[:,2]-z_crop[0], c='linen', s=dot_s/10, alpha=0.8)
ax2.scatter(coords_vis_pooled[mp_ind_pooled,0]-x_crop[0], coords_vis_pooled[mp_ind_pooled,2]-z_crop[0], c=colors_vis_pooled[mp_ind_pooled]/255, s=dot_s/10, alpha=0.8)

#Scale bars
ax1.plot((scale_bar_xpos, scale_bar_xpos+scale_bar_len), (scale_bar_ypos1, scale_bar_ypos1), c='black')
ax1.text(scale_bar_xpos+(scale_bar_len/2), scale_bar_ypos1+10, r'{}$\mu$m'.format(scale_bar_len), va='top', ha='center', fontsize=fs)

for ax in [ax1, ax2, ax3]:
    ax.set_aspect('equal')
    ax.axis('off')
    
ax1.set_xlim([0, x_crop[1]-x_crop[0]])
ax1.set_ylim([0, y_crop[1]-y_crop[0]])

ax2.set_xlim([0, x_crop[1]-x_crop[0]])
ax2.set_ylim([0, z_crop[1]-z_crop[0]])

ax3.set_xlim([0, z_crop[1]-z_crop[0]])
ax3.set_ylim([0, y_crop[1]-y_crop[0]])

ax1.invert_yaxis()

#Scale bars
ax1.plot((scale_bar_xpos, scale_bar_xpos+scale_bar_len), (scale_bar_ypos1, scale_bar_ypos1), c='black')
plt.subplots_adjust(hspace=0.2, wspace=0.01, left=0.05, right=0.95, top=0.95, bottom=0.05)
